In [ ]:
# average / weighted_average / bitwise_or
ENSEMBLE_METHOD = "bitwise_or"
VIABLE_COUNTABILITY = 0
AVERAGE_ACCEPTABILITY = 0
# Minimum value of credibility per mask
CREDIBILITY_THRESHOLD = 0.5
# Minimum IoU in order to group instances together
DEVIATION_THRESHOLD = 0.5
# Minimum IoU in order to compare instances while evaluating 
DEVIATION_THRESHOLD_EVAL = 0.5
#For Weighted Average
AVERAGE_ACCEPTABILITY_2 = 0.5

In [ ]:
import os

model_dict = []
model_dict.append(('hybrid_task_cascade_mask_rcnn_X-101-64x4d-FPN',('configs/htc/htc_x101_64x4d_fpn_dconv_c3-c5_mstrain_400_1400_16x1_20e_coco.py',
                                                                'checkpoints/htc_x101_64x4d_fpn_dconv_c3-c5_mstrain_400_1400_16x1_20e_coco_20200312-946fd751.pth',
                                                                'https://download.openmmlab.com/mmdetection/v2.0/htc/htc_x101_64x4d_fpn_dconv_c3-c5_mstrain_400_1400_16x1_20e_coco/htc_x101_64x4d_fpn_dconv_c3-c5_mstrain_400_1400_16x1_20e_coco_20200312-946fd751.pth')))
model_dict.append(('detectors_htc_r101_20e_coco',('configs/detectors/detectors_htc_r101_20e_coco.py',
                                                  'checkpoints/detectors_htc_r101_20e_coco_20210419_203638-348d533b.pth',
                                                  'https://download.openmmlab.com/mmdetection/v2.0/detectors/detectors_htc_r101_20e_coco/detectors_htc_r101_20e_coco_20210419_203638-348d533b.pth')))
model_dict.append(('cascade_mask_rcnn_X-101-64x4d-FPN',('configs/cascade_rcnn/cascade_mask_rcnn_x101_64x4d_fpn_mstrain_3x_coco.py',
                                                        'checkpoints/cascade_mask_rcnn_x101_64x4d_fpn_mstrain_3x_coco_20210719_210311-d3e64ba0.pth',
                                                        'https://download.openmmlab.com/mmdetection/v2.0/cascade_rcnn/cascade_mask_rcnn_x101_64x4d_fpn_mstrain_3x_coco/cascade_mask_rcnn_x101_64x4d_fpn_mstrain_3x_coco_20210719_210311-d3e64ba0.pth')))
model_dict.append(('cascade_mask_rcnn_x101_32x4d_fpn_dconv_c3-c5_1x_coco',('configs/dcn/cascade_mask_rcnn_x101_32x4d_fpn_dconv_c3-c5_1x_coco.py',
                                                                           'checkpoints/cascade_mask_rcnn_x101_32x4d_fpn_dconv_c3-c5_1x_coco-e75f90c8.pth',
                                                                           'https://download.openmmlab.com/mmdetection/v2.0/dcn/cascade_mask_rcnn_x101_32x4d_fpn_dconv_c3-c5_1x_coco/cascade_mask_rcnn_x101_32x4d_fpn_dconv_c3-c5_1x_coco-e75f90c8.pth')))
model_dict.append(('gcnet_X-101-FPN_DCN_Cascade_Mask_GC(c3-c5,r4)',('configs/gcnet/cascade_mask_rcnn_x101_32x4d_fpn_syncbn-backbone_dconv_c3-c5_r4_gcb_c3-c5_1x_coco.py',
                                                                    'checkpoints/cascade_mask_rcnn_x101_32x4d_fpn_syncbn-backbone_dconv_c3-c5_r4_gcb_c3-c5_1x_coco_20210615_161851-720338ec.pth',
                                                                    'https://download.openmmlab.com/mmdetection/v2.0/gcnet/cascade_mask_rcnn_x101_32x4d_fpn_syncbn-backbone_dconv_c3-c5_r4_gcb_c3-c5_1x_coco/cascade_mask_rcnn_x101_32x4d_fpn_syncbn-backbone_dconv_c3-c5_r4_gcb_c3-c5_1x_coco_20210615_161851-720338ec.pth')))

#test_config = 'configs/common/mstrain-poly_3x_coco_instance.py'
test_config = 'configs/common/mstrain-poly_3x_3000_coco_instance.py'
#test_config = 'configs/_base_/datasets/cityscapes_instance.py'
#test_config = 'configs/common/mstrain-poly_3x_coco_instance_train.py'
dataset_name = os.path.splitext(test_config)[0].split('/')[-1]

In [ ]:
import numpy as np
import torch

def IoU(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

def IoU_Mask(maskA, maskB):
    intersection = np.logical_and(maskA, maskB).astype(np.uint8)
    union = np.logical_or(maskA, maskB).astype(np.uint8)
    iou = np.sum(intersection)/np.sum(union)
    return iou

In [ ]:
from mmcv import Config
from mmdet.datasets import build_dataset, build_dataloader
from mmcv.parallel import MMDataParallel
from mmdet.apis import single_gpu_test
from mmdet.models import build_detector
from mmcv.runner import load_checkpoint
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
from typing import List, Tuple, Union, Dict
from torch import nn
from os import path
from urllib import request
from tqdm import tqdm
from PIL import Image

import mmcv
import os.path as osp
import pycocotools.mask as mask_util

COCO_CLASSES = ('person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
                'train', 'truck', 'boat', 'traffic light', 'fire hydrant',
                'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog',
                'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe',
                'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
                'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat',
                'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
                'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
                'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot',
                'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
                'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop',
                'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',
                'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock',
                'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush')

CITYSCAPES_CLASSES = ('person', 'rider', 'car', 'truck', 'bus', 'train', 'motorcycle',
                      'bicycle')

WORK_DIR = "work_dirs/ensemble_results/"

def inference_on_dataset(model_info):
    config, checkpoint = model_info
    
    cfg = Config.fromfile(config)

    if cfg.get('custom_imports', None):
        from mmcv.utils import import_modules_from_strings
        import_modules_from_strings(**cfg['custom_imports'])
    # set cudnn_benchmark
    if cfg.get('cudnn_benchmark', False):
        torch.backends.cudnn.benchmark = True

    cfg.model.pretrained = None
    if cfg.model.get('neck'):
        if isinstance(cfg.model.neck, list):
            for neck_cfg in cfg.model.neck:
                if neck_cfg.get('rfp_backbone'):
                    if neck_cfg.rfp_backbone.get('pretrained'):
                        neck_cfg.rfp_backbone.pretrained = None
        elif cfg.model.neck.get('rfp_backbone'):
            if cfg.model.neck.rfp_backbone.get('pretrained'):
                cfg.model.neck.rfp_backbone.pretrained = None
                
    test_cfg = Config.fromfile(test_config)
    # in case the test dataset is concatenated
    samples_per_gpu = 1
    if isinstance(cfg.data.test, dict):
        test_cfg.data.test.test_mode = True
        samples_per_gpu = test_cfg.data.test.pop('samples_per_gpu', 1)
        if samples_per_gpu > 1:
            # Replace 'ImageToTensor' to 'DefaultFormatBundle'
            test_cfg.data.test.pipeline = replace_ImageToTensor(
                test_cfg.data.test.pipeline)
    elif isinstance(test_cfg.data.test, list):
        for ds_cfg in test_cfg.data.test:
            ds_cfg.test_mode = True
        samples_per_gpu = max(
            [ds_cfg.pop('samples_per_gpu', 1) for ds_cfg in test_cfg.data.test])
        if samples_per_gpu > 1:
            for ds_cfg in test_cfg.data.test:
                ds_cfg.pipeline = replace_ImageToTensor(ds_cfg.pipeline)

    # init distributed env first, since logger depends on the dist info.
    distributed = False

    #rank, _ = get_dist_info()
    # allows not to create
    #mmcv.mkdir_or_exist(osp.abspath(args.work_dir))
    #timestamp = time.strftime('%Y%m%d_%H%M%S', time.localtime())
    #json_file = osp.join(args.work_dir, f'eval_{timestamp}.json')

    # build the dataloader
    dataset = build_dataset(test_cfg.data.test)
    
    data_loader = build_dataloader(
        dataset,
        samples_per_gpu=samples_per_gpu,
        workers_per_gpu=test_cfg.data.workers_per_gpu,
        dist=distributed,
        shuffle=False)

    # build the model and load checkpoint
    cfg.model.train_cfg = None
    model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
    fp16_cfg = cfg.get('fp16', None)
    if fp16_cfg is not None:
        wrap_fp16_model(model)
    checkpoint = load_checkpoint(model, checkpoint, map_location='cpu')
    #if args.fuse_conv_bn:
        #model = fuse_conv_bn(model)
    # old versions did not save class info in checkpoints, this walkaround is
    # for backward compatibility
    if 'CLASSES' in checkpoint.get('meta', {}):
        model.CLASSES = checkpoint['meta']['CLASSES']
    else:
        model.CLASSES = dataset.CLASSES

    #classes = model.CLASSES.copy()
    classes = list(enumerate(model.CLASSES)).copy()
    model = MMDataParallel(model, device_ids=[0])
    outputs = single_gpu_test(model, data_loader, None, None, None)
    
    print(len(dataset))
    
    return outputs, classes, len(dataset)


def gather_results_from_model(model_name: str, model_info: Tuple[str,str], score_thr:float, person_only:bool , result_type = 'bbox'):
    if not osp.exists(WORK_DIR+model_name+"_"+dataset_name+".pkl"):
        mmcv.mkdir_or_exist(osp.abspath(WORK_DIR))
        results, original_classes, dataset_size = inference_on_dataset(model_info)
        classes = original_classes
        mmcv.dump(results, WORK_DIR+model_name+"_"+dataset_name+".pkl")
    else:
        config,checkpoint = model_info
        cfg = Config.fromfile(config)
        model = build_detector(cfg.model)
        checkpoint = load_checkpoint(model, checkpoint, map_location='cpu')
        if 'CLASSES' in checkpoint.get('meta', {}):
            model.CLASSES = checkpoint['meta']['CLASSES']
        else:
            model.CLASSES = dataset.CLASSES
        original_classes = list(enumerate(model.CLASSES)).copy()
        classes = original_classes
        results = mmcv.load(WORK_DIR+model_name+"_"+dataset_name+".pkl")
        dataset_size = len(results)
    if person_only:
        if len(classes) == len(COCO_CLASSES):
            classes = [(0,'person')]
        elif len(classes) == len(CITYSCAPES_CLASSES):
            classes = [(0,'person'),(1,'rider')]
    
        
    return results,classes,dataset_size

def gather_results(model_dict: Dict[str,Tuple[str,str,str]], score_thr: float, person_only: bool, result_type='bbox'):
    #model_dict = model_dict.items()
    ensemble_results = {}
    dataset_compatible = -1
    label_type = []
    for i, (name, (config,checkpoint,download_link)) in enumerate(model_dict):
        if not path.exists(checkpoint):
            print("Downloading",name)
            request.urlretrieve(download_link,checkpoint)
            print("Finished downloading",name)
        print("Loading inference results from model:",name)
        ensemble_results[i],classes,dataset_size = gather_results_from_model(name, (config,checkpoint), score_thr, person_only, result_type)
        label_type.append(len(classes))
        if dataset_compatible < 0 or dataset_compatible == dataset_size:
            dataset_compatible = dataset_size
        else:
            raise(Exception("Dataset sizes are not compatible"))
    return ensemble_results,classes,dataset_compatible

def generate_dataset(dataset,model_dict,ensemble_results, labels: List[str], dataset_size, score_thr, threshold, ensemble_method):
    #ensemble_results[model][image][bbox or segm][label][instance]
    final_results = []
    n_models = len(ensemble_results)
    #Iterate over all the images
    for img in tqdm(range(0,dataset_size)):
        bbox_group = []
        segm_group = []
        ensemble_group = []
        img_results = []

        #Iterate over all the labels
        for (label_nr,label) in labels:
            bbox_results = []
            segm_results = []
            ensemble_res = []
            #Create a matrix of already used instances
            used_instances = []
            for cur_model in range(0,len(ensemble_results)):
                used_instances.insert(cur_model,[False]*len(ensemble_results[cur_model][img][0][label_nr]))
                
            #Iterate over all the models for a certain label and a certain image
            for cur_model in range(0,len(ensemble_results)):
                #Iterate over the current model's results on a certain label on a certain image
                for cur_instance in range(0,len(ensemble_results[cur_model][img][0][label_nr])):
                    if not used_instances[cur_model][cur_instance] and ensemble_results[cur_model][img][0][label_nr][cur_instance][4] >= CREDIBILITY_THRESHOLD:
                    #if not used_instances[cur_model][cur_instance] and ensemble_results[cur_model][img][0][label_nr][cur_instance][4] >= model_dict[cur_model][1][3]:
                        used_instances[cur_model][cur_instance] = True
                        cur_instance_group = [None for w in range(0,len(ensemble_results))]
                        cur_instance_group[cur_model] = (ensemble_results[cur_model][img][0][label_nr][cur_instance],
                                                         ensemble_results[cur_model][img][1][label_nr][cur_instance])
                        #Iterate over all the other models
                        for comp_model in range(cur_model+1,len(ensemble_results)):
                            deviations = []
                            #Iterate over each of the other model's results
                            for comp_instance in range(0,len(ensemble_results[comp_model][img][0][label_nr])):
                                if ensemble_results[comp_model][img][0][label_nr][comp_instance][4] >= CREDIBILITY_THRESHOLD:
                                #if ensemble_results[comp_model][img][0][label_nr][comp_instance][4] >= model_dict[comp_model][1][3]:
                                    if not used_instances[comp_model][comp_instance]:
                                        #cur_iou = IoU(ensemble_results[cur_model][img][0][label_nr][cur_instance],ensemble_results[comp_model][img][0][label_nr][comp_instance])
                                        boxA = ensemble_results[cur_model][img][0][label_nr][cur_instance]
                                        boxB = ensemble_results[comp_model][img][0][label_nr][comp_instance]
                                        xA = int(round(min(boxA[0], boxB[0])))
                                        yA = int(round(min(boxA[1], boxB[1])))
                                        xB = int(round(max(boxA[2], boxB[2])))
                                        yB = int(round(max(boxA[3], boxB[3])))
                                        cur_iou = IoU_Mask(mask_util.decode(ensemble_results[cur_model][img][1][label_nr][cur_instance])[yA:yB,xA:xB],
                                                           mask_util.decode(ensemble_results[comp_model][img][1][label_nr][comp_instance])[yA:yB,xA:xB])
                                        
                                    else:
                                        cur_iou = 0.0
                                    deviations.append(cur_iou)
                            #Check if the max iou is within the threshold and add the new instance to the group
                            if len(deviations) > 0:
                                pos = max(range(len(deviations)), key=deviations.__getitem__)
                                if deviations[pos] >= threshold:
                                    #Guarantee this instance isn't used again
                                    used_instances[comp_model][pos] = True
                                    cur_instance_group[comp_model] = (ensemble_results[comp_model][img][0][label_nr][pos],
                                                                      ensemble_results[comp_model][img][1][label_nr][pos])
                        
                        count = 0
                        for instance_i in cur_instance_group:
                            if instance_i:
                                count += 1
                                
                        # Assuming an instance group is viable if most of the networks identified it
                        if (count >= (n_models/2) + VIABLE_COUNTABILITY and (not (ensemble_method == "bitwise_and")) and (not (ensemble_method == "bitwise_or"))) or \
                           (count == n_models and ensemble_method == "bitwise_and") or \
                           (ensemble_method == "bitwise_or"):
                            bbox = np.array([0.0]*5)
                            for model_result in range(0,len(cur_instance_group)):
                                if not cur_instance_group[model_result] is None:
                                    bbox = np.add(bbox,cur_instance_group[model_result][0])
                            bbox = (bbox/count)
                            confidence = bbox[4]
                            bbox = bbox.astype(int)
                            bbox[0:3] = np.around(bbox[0:3])
                            bbox_y = (bbox[3]-bbox[1]).astype(int)
                            bbox_x = (bbox[2]-bbox[0]).astype(int)
                            mask = np.zeros((bbox_y,bbox_x),dtype=int)
                            img_size = (0,0)
                            if ensemble_method == "average":
                                for model_result in range(0,len(cur_instance_group)):
                                    if not cur_instance_group[model_result] is None:
                                        decoded_mask = mask_util.decode(cur_instance_group[model_result][1])
                                        mask = mask+decoded_mask[bbox[1]:bbox[1]+bbox_y,bbox[0]:bbox[0]+bbox_x].astype(int)
                                        img_size = decoded_mask.shape
                                acceptability = max(1,count/2 + AVERAGE_ACCEPTABILITY)
                                mask = mask >= acceptability
                            elif ensemble_method == "weighted_average":
                                total_confidence = 0.0
                                for model_result in range(0,len(cur_instance_group)):
                                    if not cur_instance_group[model_result] is None:
                                        decoded_mask = mask_util.decode(cur_instance_group[model_result][1])
                                        mask = mask+(decoded_mask[bbox[1]:bbox[1]+bbox_y,bbox[0]:bbox[0]+bbox_x].astype(int) * confidence)
                                        total_confidence += confidence
                                        img_size = decoded_mask.shape
                                mask = mask >= AVERAGE_ACCEPTABILITY_2 * total_confidence
                            elif ensemble_method == "bitwise_or":
                                for model_result in range(0,len(cur_instance_group)):
                                    if not cur_instance_group[model_result] is None:
                                        decoded_mask = mask_util.decode(cur_instance_group[model_result][1])
                                        mask = mask+decoded_mask[bbox[1]:bbox[1]+bbox_y,bbox[0]:bbox[0]+bbox_x].astype(int)
                                        img_size = decoded_mask.shape
                                mask = mask > 0.0
                            elif ensemble_method == "bitwise_and":
                                for model_result in range(0,len(cur_instance_group)):
                                    decoded_mask = mask_util.decode(cur_instance_group[model_result][1])
                                    mask = mask+decoded_mask[bbox[1]:bbox[1]+bbox_y,bbox[0]:bbox[0]+bbox_x].astype(int)
                                    img_size = decoded_mask.shape
                                mask = mask == float(n_models)
                            segmentation = np.zeros(img_size).astype(bool)
                            segmentation[bbox[1]:bbox[1]+bbox_y,bbox[0]:bbox[0]+bbox_x] = mask
                            bbox = bbox.astype(float)
                            bbox[4] = confidence
                            img_results.append((np.array(bbox),mask_util.encode(np.asfortranarray(segmentation)),np.array(cur_instance_group)))
                            #bbox_results.append(np.array(bbox))
                            #segm_results.append(mask_util.encode(np.asfortranarray(segmentation)))
                            #ensemble_res.append(np.array(cur_instance_group))
            #if not bbox_results is None:
                #np.append(bbox_results,np.array([]))  
            #bbox_group.append(np.array(bbox_results).reshape(-1,5)) 
            #segm_group.append(segm_results)
            #ensemble_group.append(ensemble_res)
        #final_results.append((bbox_group,segm_group,ensemble_group))    
        final_results.append(img_results)
    
    return final_results

def get_dataset():
    test_cfg = Config.fromfile(test_config)
    # in case the test dataset is concatenated
    samples_per_gpu = 1
    if isinstance(test_cfg.data.test, dict):
        test_cfg.data.test.test_mode = True
        samples_per_gpu = test_cfg.data.test.pop('samples_per_gpu', 1)
        if samples_per_gpu > 1:
            # Replace 'ImageToTensor' to 'DefaultFormatBundle'
            test_cfg.data.test.pipeline = replace_ImageToTensor(
                test_cfg.data.test.pipeline)
    elif isinstance(test_cfg.data.test, list):
        for ds_cfg in test_cfg.data.test:
            ds_cfg.test_mode = True
        samples_per_gpu = max(
            [ds_cfg.pop('samples_per_gpu', 1) for ds_cfg in test_cfg.data.test])
        if samples_per_gpu > 1:
            for ds_cfg in test_cfg.data.test:
                ds_cfg.pipeline = replace_ImageToTensor(ds_cfg.pipeline)


    # init distributed env first, since logger depends on the dist info.
    distributed = False

    #rank, _ = get_dist_info()
    # allows not to create
    #mmcv.mkdir_or_exist(osp.abspath(args.work_dir))
    #timestamp = time.strftime('%Y%m%d_%H%M%S', time.localtime())
    #json_file = osp.join(args.work_dir, f'eval_{timestamp}.json')

    # build the dataloader
    dataset = build_dataset(test_cfg.data.test)
    data_loader = build_dataloader(
        dataset,
        samples_per_gpu=samples_per_gpu,
        workers_per_gpu=test_cfg.data.workers_per_gpu,
        dist=distributed,
        shuffle=False)
    
    return dataset, data_loader


def get_results(model_dict, score_thr, person_only, ensemble_method, dataset, result_type='segm'):
    ensemble_results,classes,dataset_size = gather_results(model_dict,score_thr,person_only,result_type)
    results = generate_dataset(dataset,model_dict,ensemble_results,classes,dataset_size,score_thr,DEVIATION_THRESHOLD,ensemble_method)
    return results


In [ ]:
import matplotlib.pyplot as plt

def match_instances(gt_segm,
                    dt_segm,
                    img_size,
                    threshold):
    matched = []
    dt_unmatched = []
    used = [False for i in range(0,len(dt_segm))]
    
    #Case Empty
    if len(gt_segm) == 0 or len(dt_segm) == 0:
        
        matched = []
        dt_unmatched = []
        for i in range(0,len(dt_segm)):
            return_group = []
            for x in dt_segm[i][2]:
                if x is None:
                    return_group.append(np.zeros(img_size,dtype=np.uint8))
                else:
                    return_group.append(mask_util.decode(x[1]))
            pred_stack = np.dstack(return_group)    
            dt_unmatched.append((pred_stack,np.zeros(img_size,dtype=np.uint8)))
        return matched, dt_unmatched

    crowds = []
    for gt in range(0,len(gt_segm)):
        if gt_segm[gt]['iscrowd'] == 1:
            crowds.append(gt)
        else:
            gt_bbox = gt_segm[gt]['bbox'].copy()
            gt_bbox[2] = gt_bbox[0]+gt_bbox[2]
            gt_bbox[3] = gt_bbox[1]+gt_bbox[3]
            gt_segm_decoded = mask_util.decode(gt_segm[gt]['segmentation'])
            deviations = []
            for dt in range(0,len(dt_segm)):
                if not used[dt]:
                    dt_bbox = dt_segm[dt][0].copy()
                    dt_bbox[2] = dt_bbox[0]+dt_bbox[2]
                    dt_bbox[3] = dt_bbox[1]+dt_bbox[3]
                    #cur_iou = IoU(gt_bbox,dt_bbox)

                    xA = int(round(min(gt_bbox[0], dt_bbox[0])))
                    yA = int(round(min(gt_bbox[1], dt_bbox[1])))
                    xB = int(round(max(gt_bbox[2], dt_bbox[2])))
                    yB = int(round(max(gt_bbox[3], dt_bbox[3])))
                    if gt_segm_decoded.shape != mask_util.decode(dt_segm[dt][1]).shape:
                        print(yA,yB,xA,xB)
                        print(gt_segm_decoded.shape,mask_util.decode(dt_segm[dt][1]).shape)
                    
                    cur_iou = IoU_Mask(gt_segm_decoded[yA:yB,xA:xB],
                                       mask_util.decode(dt_segm[dt][1])[yA:yB,xA:xB])
                    deviations.append(cur_iou)
                else:
                    deviations.append(0.0)
            pos = max(range(len(deviations)), key=deviations.__getitem__)
            if deviations[pos] >= threshold:
                #Guarantee this instance isn't used again
                used[pos] = True
                return_group = []
                for x in dt_segm[pos][2]:
                    if x is None:
                        return_group.append(np.zeros(img_size,dtype=np.uint8))
                    else:
                        return_group.append(mask_util.decode(x[1]))
                pred_stack = np.dstack(return_group)
                if pred_stack.shape[2] != 5:
                    print(pred_stack.shape)
                matched.append((pred_stack,mask_util.decode(gt_segm[gt]['segmentation'])))
    """
    for i in range(0,len(used)):
        if not used[i]:
            dt_segm_decoded = mask_util.decode(dt_segm[dt][1])
            dt_bbox = dt_segm[dt][0].copy()
            dt_bbox[2] = dt_bbox[0]+dt_bbox[2]
            dt_bbox[3] = dt_bbox[1]+dt_bbox[3]
            xA,xB,yA,yB = int(round(dt_bbox[0])),int(round(dt_bbox[2])),int(round(dt_bbox[1])),int(round(dt_bbox[3]))
            deviations = []
            for c in crowds:
                intersection = np.logical_and(mask_util.decode(gt_segm[c]['segmentation']),
                                              dt_segm_decoded)
                cur_iou = IoU_Mask(dt_segm_decoded[yA:yB,xA:xB],
                                   intersection[yA:yB,xA:xB])
                
                fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(15,15))
                ax=ax.flat
                ax[0].set_title("Original Image")  # set title
                ax[0].imshow(image)
                ax[1].set_title("DT SEGM")  # set title
                ax[1].imshow(dt_segm_decoded,cmap='gray',vmin=0,vmax=1)
                ax[2].set_title("Intersection")  # set title
                ax[2].imshow(intersection,cmap='gray',vmin=0,vmax=1)
                ax[3].set_title("Crowd")  # set title
                ax[3].imshow(mask_util.decode(gt_segm[c]['segmentation']),cmap='gray',vmin=0,vmax=1)
                fig.tight_layout()
                plt.show()
                
                deviations.append(cur_iou)
            if len(crowds) != 0:
                pos = max(range(len(deviations)), key=deviations.__getitem__)
                if deviations[pos] < threshold:
                    return_group = []
                    for x in dt_segm[i][2]:
                        if x is None:
                            return_group.append(np.zeros(img_size,dtype=np.uint8))
                        else:
                            return_group.append(mask_util.decode(x[1]))
                    pred_stack = np.dstack(return_group)    
                    dt_unmatched.append((pred_stack,np.zeros(img_size,dtype=np.uint8)))
            else:
                return_group = []
                for x in dt_segm[i][2]:
                    if x is None:
                        return_group.append(np.zeros(img_size,dtype=np.uint8))
                    else:
                        return_group.append(mask_util.decode(x[1]))
                pred_stack = np.dstack(return_group)    
                dt_unmatched.append((pred_stack,np.zeros(img_size,dtype=np.uint8)))

    """

    return matched, dt_unmatched

def matcher_no_img(dt,dataset,img=False):
    result_dataset = []
    gt = dataset.gt_return()
    #print(gt.keys())
    for i in tqdm(range(0,len(dataset))):        
        filename = dataset[i]['img_metas'][0].data['filename']
        size = dataset[i]['img_metas'][0].data['ori_shape']
        
        imgId = filename.split('/')[-1].split('.')[0]
        cur_dt = dt[i]
        cur_gt = gt[int(imgId)]
        
        #image = Image.open(filename)
        #img_array = np.asarray(image)
        
        matched, unmatched = match_instances(cur_gt,cur_dt,(size[0],size[1]),DEVIATION_THRESHOLD_EVAL)
        for m in matched:
            res = m[0].copy()
            result_dataset.append((filename,True,res,m[1]))
        """
        for u in unmatched:
            res = u[0].copy()
            result_dataset.append((filename,False,res,u[1]))
        """
            
    return result_dataset

def matcher(dt,dataset):
    result_dataset = []
    gt = dataset.gt_return()
    for i in tqdm(range(0,len(dataset))):
        filename = dataset[i]['img_metas'][0].data['filename']
        size = dataset[i]['img_metas'][0].data['ori_shape']
        
        imgId = filename.split('/')[-1].split('.')[0]
        cur_dt = dt[i]
        cur_gt = gt[int(imgId)]
        image = Image.open(filename)
        img_array = np.asarray(image)
        
        matched, unmatched = match_instances(cur_gt,cur_dt,(size[0],size[1]),DEVIATION_THRESHOLD_EVAL)
        for m in matched:
            res = np.dstack((img_array,m[0]))
            result_dataset.append((filename,True,res,m[1]))
        """for u in unmatched:
            res = np.dstack((img_array,u[0]))
            result_dataset.append((filename,res,u[1]))"""
            
    return result_dataset
        


In [ ]:
import glob
import warnings
import gc
import os
import pickle

warnings.filterwarnings('ignore')
dataset, data_loader = get_dataset()
results = get_results(model_dict, CREDIBILITY_THRESHOLD,True,ENSEMBLE_METHOD,dataset,result_type='segm')
warnings.filterwarnings('default')

In [ ]:
print(len(results))

In [ ]:
dataset, data_loader = get_dataset()
final_results = matcher(results,dataset)

In [ ]:
print(len(final_results))

In [ ]:
import cv2
import matplotlib.pyplot as plt

count = 0

for (filename,_,dt_segm,gt_segm) in tqdm(final_results):
    if dt_segm.shape[2] != 8:
        img = cv2.cvtColor(dt_segm[:,:,0],cv2.COLOR_GRAY2RGB)
        dt = dt_segm[:,:,1:]
    else:
        img = dt_segm[:,:,0:3]
        dt = dt_segm[:,:,3:]
    gt = gt_segm
    
    cv2.imwrite("data/coco_bitwise_or_reduced_ensemble_results/img/"+str(count)+".png",img)
    cv2.imwrite("data/coco_bitwise_or_reduced_ensemble_results/net1/"+str(count)+".png",dt[:,:,0]*255)
    cv2.imwrite("data/coco_bitwise_or_reduced_ensemble_results/net2/"+str(count)+".png",dt[:,:,1]*255)
    cv2.imwrite("data/coco_bitwise_or_reduced_ensemble_results/net3/"+str(count)+".png",dt[:,:,2]*255)
    cv2.imwrite("data/coco_bitwise_or_reduced_ensemble_results/net4/"+str(count)+".png",dt[:,:,3]*255)
    cv2.imwrite("data/coco_bitwise_or_reduced_ensemble_results/net5/"+str(count)+".png",dt[:,:,4]*255)
    cv2.imwrite("data/coco_bitwise_or_reduced_ensemble_results/gt/"+str(count)+".png",gt*255)
    count += 1

In [ ]:
import pickle

with open('work_dirs/dataset_generation/reduced_coco_train_average_img.pkl', 'wb') as handle:
    #compressed_file = bz2.BZ2File(handle, 'w')
    pickle.dump(final_results,handle)

In [ ]:
import pickle

with open('work_dirs/dataset_generation/coco_average_no_img.pkl', 'rb') as handle:
    #compressed_file = bz2.BZ2File(handle, 'w')
    results = pickle.load(handle)

In [ ]:
print(len(final_results))

In [ ]:
for img in range(0,len(final_results)):
    if not final_results[img][1]:
        print(final_results[img])

In [ ]:
import numpy as np
import scipy.signal
from multiprocessing import Pool
from enum import Enum

class Direction(Enum):
    N = 0
    NE = 1
    E = 2
    SE = 3
    S = 4
    SW = 5
    W = 6
    NW = 7

transform = {
    'N': (0,-1),
    'NE': (1,-1),
    'E': (1,0),
    'SE': (1,1),
    'S': (0,1),
    'SW': (-1,1),
    'W': (-1,0),
    'NW': (-1,-1)
}

"""def _is_border(kernel):
    if kernel[0][1] and kernel[1][0] and kernel[2][1] and kernel[1][2]:
        return True
    else:
        return False"""

# [[0,1,0],
#  [1,1,1],
#  [0,1,0]]

def _form_boundary_mask(mask):
    padded_mask = np.pad(mask, [(32,32),(32,32)], mode='constant', constant_values=[(0,0),(0,0)])
    padded_mask_shape = padded_mask.shape
    #boundary_mask = np.zeros(mask.shape,dtype=np.bool_)
    kernel = [[0,1,0],
              [1,5,1],
              [0,1,0]]
    boundary_mask = scipy.signal.convolve2d(padded_mask,kernel,boundary="fill",fillvalue=0.0)
    boundary_mask = (boundary_mask < 9.0) & (boundary_mask > 5.0)
    
    return boundary_mask

def _transform_dir(direction):
    return transform[Direction(direction).name]

def _find_next_dir(boundary_mask,cur_pixel,last_dir):
    next_dir = Direction((Direction[last_dir].value-3)%8).name
    inv_dir = Direction((Direction[last_dir].value-4)%8).name
    
    while True:
        next_pixel = [sum(x) for x in zip(cur_pixel,transform[next_dir])]
        if next_dir == inv_dir:
            return None
        elif boundary_mask[next_pixel[1]][next_pixel[0]] == 1:
            return next_pixel,next_dir
        else:
            next_dir = Direction((Direction[next_dir].value+1)%8).name
    
        

def _find_highest_pixel(boundary):
    if len(np.where(boundary == 1)[0]) > 0:
        y = min(np.where(boundary == 1)[0])
        x = min(np.where(boundary[y] == 1)[0])
        return (x,y)
    else:
        return None
    

def _order_boundary_pixels(boundary):    
    ordered_pixels = []
    
    exists_bounds = True

    cur_pixel = _find_highest_pixel(boundary)
    plt.figure(figsize=(20, 20))
    plt.imshow(boundary,cmap='gray')
    plt.show()
    cv2.imwrite("../boundary.png",boundary*255)
    
    while exists_bounds:
        cur_list = []
        
        cur_list.append(cur_pixel)
        last_dir = "E"
        exists_next = True 

        while exists_next:
            boundary[cur_pixel[1],cur_pixel[0]] = 0.0

            res = _find_next_dir(boundary,cur_pixel,last_dir)
            if not res:
                exists_next = False
            else:
                cur_pixel,last_dir = res
                cur_list.append(cur_pixel)        
        
        ordered_pixels.append(cur_list)
        cur_pixel = _find_highest_pixel(boundary)
        if not cur_pixel:
            exists_bounds = False
    
    return ordered_pixels

def gen_sliding_window(mask):
    boundary_mask = _form_boundary_mask(mask)
    temp_mask = boundary_mask.copy()
    padded_mask = np.pad(mask.copy(), [(32,32),(32,32)], mode='constant', constant_values=[(0,0),(0,0)])
    print(padded_mask.shape)
    sliding_window = _order_boundary_pixels(temp_mask)
    
    for group in sliding_window:
        count = 0
        
        for pixel in group[::32]:
            print(pixel)
            square = padded_mask[pixel[1]-31:pixel[1]+32,pixel[0]-31:pixel[0]+32]
            plt.figure(figsize=(4, 4))
            plt.imshow(square,cmap='gray')
            plt.show()
            

    return sliding_window

In [ ]:
import cv2
import matplotlib.pyplot as plt

for (n,(filename,_,dt_segm,gt_segm)) in enumerate(final_results,0):
    print(dt_segm.shape)
    plt.figure(figsize=(20, 20))
    plt.imshow(dt_segm[:,:,0:3])
    plt.show()

"""img = cv2.imread("../img.png",cv2.IMREAD_GRAYSCALE)
#img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
print(img.shape)
plt.figure(figsize=(20, 20))
plt.imshow(img,cmap='gray')
plt.show()
windows = gen_sliding_window(img/255)"""

In [ ]:
import pickle
import bz2

with open('work_dirs/dataset_generation/cityscapes_full.pkl', 'wb') as handle:
    #compressed_file = bz2.BZ2File(handle, 'w')
    pickle.dump(final_results, handle)

In [ ]:
len(final_results)

In [ ]:
import json

print("Loading json...")

f = open('data/coco/annotations/instances_val2017.json')

data = json.load(f)

print("Json Correctly Loaded")

f.close()

In [ ]:
print(list(data.keys()))

In [ ]:
print(data['images'][0])

In [ ]:
id_arr = []
has_crowd = []

#for im in data["images"]:
    #id_arr.append(im['id'])
#has_crowd = list.fromkeys(id_arr)

#isolate annotations only referring to people
only_people_ann = [x for x in data["annotations"] if x["category_id"] == 1]

#mark crowded images
images_with_crowds = [x["image_id"] for x in only_people_ann if x["iscrowd"] == 1]

#remove images that have crowds
final_images = [x for x in data["images"] if not x["id"] in images_with_crowds]

#remove annotations referring to images that have crowds
final_annotations = [x for x in only_people_ann if not x["image_id"] in images_with_crowds]

#remove every category besides person
final_categories = [data["categories"][0]]


In [ ]:
print(len(final_images))

In [ ]:
print(len(final_annotations))

In [ ]:
print(final_categories)

In [ ]:
print(data["info"])

In [ ]:
data = {'info': data['info'],
        'licenses': data['licenses'],
        'images': final_images,
        'annotations': final_annotations,
        'categories': final_categories}

In [ ]:
with open('data/coco/annotations/instances_val2017_person_no_crowds.json', 'w') as outfile:
    json.dump(data, outfile)

In [ ]:
new_images = data['images'][0:3000]
print(len(new_images))

In [ ]:
id_arr = []
for im in new_images:
    id_arr.append(im['id'])
id_dict = dict.fromkeys(id_arr)

In [ ]:
new_annotations = []
for ann in data['annotations']:
    if ann['image_id'] in id_dict:
        new_annotations.append(ann)

In [ ]:
print(len(new_annotations))
print(len(data['annotations']))

In [ ]:
print(new_images[0])

In [ ]:
print(data['annotations'][0])